# CRFs
- 2020-2 Artifitial Intelligence
- week6 assignment 
- updated 1011

<pre>
<h2> Homework</h2>
<h2> CRFs를 이용한 자동 띄어쓰기 성능 향상</h2>
  입력 자질 : 직전 음절, 현재 음절, 다음 음절
  기존 방법
  ["나", "는", "사", "과", "가", "좋", "아"]
            ->
            [ {"WORD":"나"}, {"WORD":"는"}, {"WORD":"사"}, ... ]
    
    나 -> {"WORD":"나"}
    는 -> {"WORD":"는"}
    사 -> {"WORD":"사"}

    각 feature dictionary에 자질 추가
</pre>


In [1]:
!pip install sklearn-crfsuite

     |████████████████████████████████| 747kB 3.5MB/s 


In [2]:
cd /content/drive/My Drive/AI_assignment

/content/drive/My Drive/AI_assignment


In [3]:
# ready
import pandas as pd
import numpy as np

import sys
np.set_printoptions(threshold=sys.maxsize)
import os
import tqdm
import warnings
warnings.filterwarnings(action='ignore')

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# visualization
from matplotlib import pyplot as plt

In [4]:
# sklearn
import sklearn_crfsuite
from sklearn_crfsuite import metrics

### step1. Load data
txt 로부터 데이터를 읽고 datas에 저장한다.  
실습파일과 동일하게 진행한다.  

In [5]:
# path
file_path = 'data/spacing_data.txt'
# read
with open (file_path,'r', encoding='utf8') as inFile:
    lines = inFile.readlines()

# save
datas = []
for line in lines:
    pieces = line.strip().split('\t')
    eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
    datas.append((eumjeol_sequence,label))

### step2. Data generation
전체 데이터를 9:1 비율에 맞춰 train & test 로 나눈다.  
셔플없이 순서대로 나눈다.

In [6]:
# index
num_of_trn = int(len(datas)*0.9)
train_datas = datas[:num_of_trn]
test_datas = datas[num_of_trn:]
          
print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)))

for data in train_datas[:5]:
  print(data)

train_datas 개수 : 900
test_datas 개수 : 100
(['약', '속', '장', '소', '인', '신', '라', '호', '텔', '커', '피', '숍', '에', '재', '옥', '이', '먼', '저', '와', '기', '다', '리', '고', '있', '었', '다', '.'], ['B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['"', '야', '!', '나', '이', '든', '처', '녀', '가', '옷', '이', '라', '도', '좀', '화', '사', '한', '색', '으', '로', '입', '고', '다', '녀', '라', '.', '회', '색', '에', '다', '검', '정', '바', '바', '리', '가', '뭐', '니', '?', '"'], ['B', 'I', 'I', 'B', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'B', 'I', 'I', 'B', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I'])
(['재', '옥', '은', '전', '에', '없', '이', '정', '현', '의', '옷', '차', '림', '을', '탓', '하', '였', '다', '.'], ['B', 'I', 'I', 'B', 'I', 'B', 'I', 'B', 'I', 'I', 'B', 'I', 'I', 'I', 'B', 'I', 'I', 'I', 'I'])
(['"', '이', '게', '어', '때', '서', '?', '갑', '자', '기', '옷', '얘', '긴', '왜', '하', '니', '?

### step3. Sequence to Feature
- 음절의 자질화  
- 문장의 각 음절을 crf 모델의 입력으로 사용할 수 있도록 자질화한다.  
- 앞 음절과 뒤 음절도 자질화 한다.  
- 자질화한 데이터와 해당 데이터의 라벨을 분리하여 각 리스트에 저장   


In [7]:
# 앞 뒤 음절을 보도록 수정된 sent2feature function
def sent2feature(eumjeol_sequence):
    features = []

    for idx, eumjeol in enumerate(eumjeol_sequence):
        feature = {}
        # current
        feature['WORD'] = eumjeol
        # front & back 음절
        try:
            front_eumjeol = eumjeol_sequence[idx-1]
            feature['F_WORD'] = front_eumjeol
        except:
            pass    
        try:
            back_eumjeol = eumjeol_sequence[idx+1]
            feature['B_WORD'] = back_eumjeol
        except:
            pass

        #feature = {'WORD':current_eumjeol, 'F_WORD':front_eumjeol, 'B_WORD':back_eumjeol}
        features.append(feature)

    return features

In [8]:
train_x, train_y = [], []

for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [9]:
print(train_x[:5])
print(train_y[:5])

[[{'WORD': '약', 'F_WORD': '.', 'B_WORD': '속'}, {'WORD': '속', 'F_WORD': '약', 'B_WORD': '장'}, {'WORD': '장', 'F_WORD': '속', 'B_WORD': '소'}, {'WORD': '소', 'F_WORD': '장', 'B_WORD': '인'}, {'WORD': '인', 'F_WORD': '소', 'B_WORD': '신'}, {'WORD': '신', 'F_WORD': '인', 'B_WORD': '라'}, {'WORD': '라', 'F_WORD': '신', 'B_WORD': '호'}, {'WORD': '호', 'F_WORD': '라', 'B_WORD': '텔'}, {'WORD': '텔', 'F_WORD': '호', 'B_WORD': '커'}, {'WORD': '커', 'F_WORD': '텔', 'B_WORD': '피'}, {'WORD': '피', 'F_WORD': '커', 'B_WORD': '숍'}, {'WORD': '숍', 'F_WORD': '피', 'B_WORD': '에'}, {'WORD': '에', 'F_WORD': '숍', 'B_WORD': '재'}, {'WORD': '재', 'F_WORD': '에', 'B_WORD': '옥'}, {'WORD': '옥', 'F_WORD': '재', 'B_WORD': '이'}, {'WORD': '이', 'F_WORD': '옥', 'B_WORD': '먼'}, {'WORD': '먼', 'F_WORD': '이', 'B_WORD': '저'}, {'WORD': '저', 'F_WORD': '먼', 'B_WORD': '와'}, {'WORD': '와', 'F_WORD': '저', 'B_WORD': '기'}, {'WORD': '기', 'F_WORD': '와', 'B_WORD': '다'}, {'WORD': '다', 'F_WORD': '기', 'B_WORD': '리'}, {'WORD': '리', 'F_WORD': '다', 'B_WORD': '고'}, {'WORD':

### step4. CRF 모델 학습
Train data set을 이용해서 모델을 학습시킨다.

In [10]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(train_x,train_y)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=None,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

### step5. 모델 평가 및 결과 확인

In [11]:
def show_predict_result(test_datas, predict):
  
  # test 데이터 띄어쓰기 과정
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]
     
      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          # 첫번째 인덱스
          if(index_2 == 0):      
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          # 두번째 인덱스 부터~ 
          if(correct_labels[index_2] == "B"):
              correct_sentence += " "    # 앞에 공백추가
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "    # 예측시에도 공백추가
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

# crf 모델이 예측
predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.9000381533765738

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
출력 문장 : 사라 예보에서 한세르 비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.

정답 문장 : 이 암살행위는 국지적인 민족주의들과 세계적인 제국주의들이 충돌하는 분쟁지역에서 저질러졌다.
출력 문장 : 이암 살행 위는 국지적인 민족주의 들과 세계적인 제국주의 들이 충돌하는 분쟁 지역에서 저질러졌다.

정답 문장 : 오토만제국의 점진적인 해체는 민족주의의 독기를 발산하는 동시에 오스트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.
출력 문장 : 오토만 제국의 점진적인해체는 민족주의의 독기를 발산하는 동시에 오스 트리아, 헝가리와 독일, 영국, 프랑스의 탐욕을 자극했다.

정답 문장 : 이렇게 해서 발칸 반도의 한 외진 장소에서 벌어진 국지적인 테러 행위는 일련의 긴박한 반응을 불러 일으키면서 전 유럽에 영향을 미쳤을 뿐만 아니라 이번에는 아시아와 아프리카 식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.
출력 문장 : 이렇게 해서 발칸 반도의 한외진 장소에서 벌어 진국지적인테 러행 위는 일련의 긴박한 반응을 불러일으키면서 전유럽에 영향을 미쳤을 뿐만 아니라이 번에는 아시아와 아프리 카식민지들, 일본, 그리고 이어서 미국과 멕시코까지 끌어들였다.

정답 문장 : 전쟁의 물결이 지구상의 모든 대양으로 밀려드는 동안 캐나다인들과 미국인들, 오스트레일리아인들, 세네갈인들, 알제리인들, 모로코인들, 안남(安南)인들은 연합군 깃발을 휘날리며 유럽전선에

단순 현재 음절 하나만 봤을 때보다, 앞 뒤 음절을 하나씩 더보게 되면 성능은 90퍼센트로 약 20퍼센트 올라가게 된다.